In [19]:
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
with open("aggregated_statistics.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [4]:
shapefile_path = "data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp"

# Загрузка геоданных
world = gpd.read_file(shapefile_path)

In [11]:
# Маппинг кодов стран (ISO A2) на количество статей
article_counts = {
    country: stats["total_articles"]
    for country, stats in data.items()
}

In [12]:
# Преобразуем коды стран (ISO A2 → ISO A3, которые в shapefile)
code_map = {
    "au": "AUS", 
    "ca": "CAN", 
    "ch": "CHE", 
    "gb": "GBR", 
    "ie": "IRL", 
    "in": "IND", 
    "ph": "PHL", 
    "pk": "PAK", 
    "sg": "SGP",
    "us": "USA"
}

In [13]:
# Обратный маппинг: ISO A3 → article count
country_article_map = {}
for code2, count in article_counts.items():
    code3 = code_map.get(code2)
    if code3:
        country_article_map[code3] = count

In [30]:
world["articles"] = world["ADM0_A3"].map(country_article_map)

In [ ]:
# Построение карты
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.boundary.plot(ax=ax, linewidth=0.8, color="black")
world.plot(column="articles", ax=ax, cmap="OrRd", legend=True,
           legend_kwds={'label': "Number of Articles", 'orientation': "vertical"},
           missing_kwds={"color": "lightgrey", "label": "No data"})

ax.set_title("Geographic Coverage of News Articles by Country")
ax.axis('off')
plt.tight_layout()
plt.savefig("geographic_coverage_map.png")
plt.close()

print("Географическая карта сохранена: geographic_coverage_map.png")


🗺️ Географическая карта сохранена: geographic_coverage_map.png


In [34]:
# Присвоение средних значений тональности
world["sentiment"] = world["ADM0_A3"].map(country_article_map)

# Шаг 4: Построение карты
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.boundary.plot(ax=ax, linewidth=0.5, color="black")
world.plot(
    column="sentiment", ax=ax, cmap="RdYlGn", legend=True,
    legend_kwds={
        'label': "Средняя тональность по странам",
        'orientation': "vertical"
    },
    missing_kwds={
        "color": "lightgrey",
        "label": "Нет данных"
    }
)

ax.set_title("Geographical distribution of news sentiment", fontsize=18)
ax.axis('off')
plt.tight_layout()
plt.savefig("sentiment_map_by_country.png")
plt.close()

print("Карта сохранена: sentiment_map_by_country.png")

Карта сохранена: sentiment_map_by_country.png
